In [1]:
import sqlite3
import pandas as pd
import numpy as np
conn = sqlite3.connect('../data/raw/hackernews/csv/hackernews.sl3')
cursor = conn.cursor()

In [2]:
%%time
df = pd.read_sql_query('SELECT by, COUNT(*) as count FROM hackernews WHERE by != "" GROUP BY by', conn)

CPU times: user 20.9 s, sys: 3.91 s, total: 24.8 s
Wall time: 28.7 s


In [18]:
query = '''
        SELECT hackernews.*
        FROM hackernews JOIN (
            SELECT by, COUNT(*)
            FROM hackernews
            WHERE by != "" and type = 'comment'
            GROUP BY by
            HAVING COUNT(*) > 50
            ORDER BY COUNT(*) ASC
            LIMIT 10000
        ) t ON hackernews.by = t.by  
        WHERE type = 'comment'
'''

In [19]:
%%time
filtered = pd.read_sql_query(query, conn)
print(len(filtered))

700585
CPU times: user 47.9 s, sys: 10.3 s, total: 58.2 s
Wall time: 1min 7s


In [20]:
filtered['id'] = filtered['id'].map(lambda x: int(x))
filtered['parent'] = filtered['parent'].map(lambda x: int(float(x)))

In [21]:
filtered

,by,dead,deleted,descendants,id,kids,parent,parts,poll,retrieved_on,score,text,time,title,type,url
0,ejames,,,,2056838,,2056472,,,1502933011,,The problem is that your assessment of risks a...,1293840372,,comment,
1,dorianj,,,,2056867,[2057119],2055773,,,1502933006,,They obviously didn't do the proper due dilige...,1293841365,,comment,
2,cdmoyer,,,,2056869,,2055558,,,1502933006,,I feel like your prejudice is coloring your re...,1293841466,,comment,
3,bobz,,,,2056875,,2055037,,,1502933005,,"In brief, and only because vicious attacks on ...",1293841689,,comment,
4,cabinguy,,,,2056909,[2056968],2056777,,,1502932999,,"If you are selling ads directly (just as hard,...",1293842642,,comment,
5,ggruschow,,,,2056914,,2056818,,,1502932998,,There were tablets on the market before Macs.,1293842812,,comment,
6,p_nathan,,,,2056934,[2057959],2056871,,,1502932995,,I'm not sure of the origins of the zenny/ascet...,1293843375,,comment,
7,rlmw,,,,2056940,[2056962],2056616,,,1502932994,,Allegedly Samsung have sold 1 million Tabs wit...,1293843651,,comment,
8,rlmw,,,,2056953,[2057005],2056873,,,1502932992,,"I don't know which nook he is referring to, bu...",1293844108,,comment,
9,archon810,,,,2056975,"[2057049, 2057032]",2056709,,,1502932988,,"So, he's been coding since he was 15? So that'...",1293844825,,comment,


In [22]:
filtered.dtypes

by              object
dead            object
deleted         object
descendants     object
id               int64
kids            object
parent           int64
parts           object
poll            object
retrieved_on    object
score           object
text            object
time            object
title           object
type            object
url             object
dtype: object

In [23]:
import sys
sys.path.append('..')
from src.graphbuilder import GraphBuilder
gb = GraphBuilder()

In [24]:
gb.extend_graph(filtered)

700585it [14:19, 814.95it/s]


In [27]:
gb.graph.ecount()

36914

In [28]:
gb.graph.vcount()


10000

In [31]:
gb.graph.write_graphml('../models/full_comment_graph.graphml')